# Capstone Project

## *Recommender System*
Testing now on my own data set of country songs. Also adjusted functions here so that they can construct the playlist based on scores but then rank based on popularity of the artist.

#### Table of Contents

* [Topic 1](#topic-1)
* [Topic 2](#topic-2)

### Import Libraries & Read in Data

In [86]:
## standard imports 
import pandas as pd 
import numpy as np
import re
## visualizations
import matplotlib.pyplot as plt
import seaborn as sns

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity, pairwise_kernels

### Spotify Credentials - must be set in local environment to run
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

## options
# pd.options.display.max_rows = 4000
# pd.options.display.max_columns = 100
# pd.set_option('max_colwidth', 100)

In [31]:
### read in data
df = pd.read_csv('../data/country-20201110_151852.csv')
# track = pd.read_csv('../data/WAP.csv')
track = pd.read_csv('../data/boston.csv')

In [32]:
df.head(2)

,track_name,artist,popularity,track_id,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Forever After All,Luke Combs,86,6IBcOGPsniK3Pso1wHIhew,country,0.487,0.650,0,-5.195,1,0.0253,0.1910,0.000000,0.0933,0.456,151.964,232533,4
1,Be Like That - feat. Swae Lee & Khalid,Kane Brown,87,5f1joOtoMeyppIcJGZQvqJ,country,0.727,0.626,7,-8.415,1,0.0726,0.0469,0.000026,0.1260,0.322,86.970,191406,4


In [33]:
drop_cols = ['key', 'mode', 'time_signature', 'duration_ms']
df.drop(columns=drop_cols, inplace=True)
track.drop(columns=drop_cols, inplace=True)  ### not present in test set using here

In [34]:
track

,track_name,artist,track_id,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Boston,Dalton & the Sheriffs,4HJ7mSMtHAdU55lLjGE4zW,0.15,0.541,0.921,-5.25,0.0443,0.00052,0.0784,0.159,0.613,99.98


In [35]:
df

,track_name,artist,popularity,track_id,genre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Forever After All,Luke Combs,86,6IBcOGPsniK3Pso1wHIhew,country,0.487,0.650,-5.195,0.0253,0.1910,0.000000,0.0933,0.456,151.964
1,Be Like That - feat. Swae Lee & Khalid,Kane Brown,87,5f1joOtoMeyppIcJGZQvqJ,country,0.727,0.626,-8.415,0.0726,0.0469,0.000026,0.1260,0.322,86.970
2,One Of Them Girls,Lee Brice,80,14GwnOeC9qYEKEA6uOZepa,country,0.643,0.790,-6.079,0.0462,0.2940,0.000000,0.1070,0.800,95.987
3,7 Summers,Morgan Wallen,81,0tbjiOUl4k492KPdWZS9sy,country,0.457,0.774,-5.201,0.0336,0.4280,0.000043,0.1190,0.590,203.863
4,Whiskey Glasses,Morgan Wallen,81,6foY66mWZN0pSRjZ408c00,country,0.614,0.680,-4.580,0.0289,0.3690,0.000002,0.1150,0.707,149.959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Speakers,Sam Hunt,54,4DXfOXYv0ki7IR3P0XRcMY,country,0.594,0.459,-7.507,0.0419,0.7120,0.000000,0.1690,0.316,75.082
1996,Easy,John King,54,5f9vvN6WCezcHMVreAvR2B,country,0.683,0.580,-6.864,0.0258,0.2260,0.000003,0.1700,0.714,103.056
1997,Mr. Tambourine Man,Bob Dylan,64,3RkQ3UwOyPqpIiIvGVewuU,country,0.390,0.310,-14.545,0.0676,0.7940,0.000000,0.3790,0.584,177.515
1998,Don't Close Your Eyes,Keith Whitley,54,3eIc0XWlqNkT290DfwNyBJ,country,0.618,0.271,-13.695,0.0260,0.7230,0.000558,0.1290,0.264,78.334


In [ ]:
# pd.concat([df, track], ignore_index=True)

## Recommender Functions <a class="anchor" id="topic-1"></a>
<hr/>

In [93]:
def track_reccommender(df, track, include_pop=True):
    ID = track['track_id'].values[0]
    ### Create X data
    data = pd.concat([df, track], ignore_index=True)
    ### desired features for model (may change later)
    features = ['popularity', 'acousticness', 'danceability', 'energy', 'speechiness', 'valence']
    if include_pop==False:
        features.remove('popularity')
    X = data[features]
    ### calculate similarity matrix
    similarity_matrix = cosine_similarity(X, X)
    
    ### create mapping bwtn track ids and index
    track_id_map = pd.Series(data.index, index=data['track_id'])
    ## find index of track in dataframe
    track_index = track_id_map[ID]
    
    ### find the correct column for the track in the similarity matrix

    ### USING SERIES INSTEAD
    similarity_scores = pd.Series(similarity_matrix[track_index])
    similarity_scores.sort_values(ascending=False, inplace=True)
    
    ### SELECT TOP 15 RESULTS FOR PLAYLIST
    top_scores = similarity_scores[:15]
    top_scores_ids = data['track_id'].loc[top_scores.index]
    
    
    ### created dataframe of reccommended tracks
    rec_tracks_df = data[data['track_id'].isin(top_scores_ids.values)].copy()
    rec_tracks_df['score'] = similarity_scores
    rec_tracks_df.sort_values(by='score', ascending=False, inplace=True)

    return rec_tracks_df

def add_track_data(df, track):
    ID = track['track_id'].values[0]
    ### Create X data
    data = pd.concat([df, track], ignore_index=True)
    ### desired features for model (may change later)
    return data

def pop_track_reccommender(df, track):
    ID = track['track_id'].values[0]
    ### Create X data
    data = pd.concat([df, track], ignore_index=True)
    ### desired features for model (may change later)
    features = ['acousticness', 'danceability', 'energy', 'speechiness', 'valence']

    X = data[features]

    ### calculate similarity matrix
    similarity_matrix = cosine_similarity(X, X)
    
    ### create mapping bwtn track ids and index
    track_id_map = pd.Series(data.index, index=data['track_id'])
    ## find index of track in dataframe
    track_index = track_id_map[ID]
    
    ### find the correct column for the track in the similarity matrix
    

    ### USING SERIES INSTEAD
    similarity_scores = pd.Series(similarity_matrix[track_index])
    similarity_scores.sort_values(ascending=False, inplace=True)
    

    ### CREATE DF OF ALL SCORES
    scores_ids = data['track_id'].loc[similarity_scores.index]
    
    
    ### CREATE DF OF ALL SCORES
    rec_tracks_df = data[data['track_id'].isin(scores_ids.values)].copy()
    rec_tracks_df['score'] = similarity_scores
    rec_tracks_df.sort_values(by=['score', 'popularity'], ascending=False, inplace=True)

    return rec_tracks_df

def pop_track_reccommender2(df, track, scorer='Cosine Similarity'):
    ID = track['track_id'].values[0]
    ### Create X data
    data = pd.concat([df, track], ignore_index=True)
    ### desired features for model (may change later)
    features = ['acousticness', 'danceability', 'energy', 'speechiness', 'valence']

    X = data[features]

    ### calculate similarity matrix 
    if scorer == 'Cosine Similarity':
        # similarity_matrix = cosine_similarity(X, X)
        similiarity_matrix = pairwise_kernels(X, metric = 'cosine')
#     elif scorer == 'linear kernel':
#         similarity_matrix = cosine_similarity(X, X)
    
    ### create mapping bwtn track ids and index
    track_id_map = pd.Series(data.index, index=data['track_id'])
    ## find index of track in dataframe
    track_index = track_id_map[ID]
    
    ### find the correct column for the track in the similarity matrix
    

    ### USING SERIES INSTEAD
    similarity_scores = pd.Series(similarity_matrix[track_index])
    similarity_scores.sort_values(ascending=False, inplace=True)
    

    ### CREATE DF OF ALL SCORES
    scores_ids = data['track_id'].loc[similarity_scores.index]
    
    
    ### CREATE DF OF ALL SCORES
    rec_tracks_df = data[data['track_id'].isin(scores_ids.values)].copy()
    rec_tracks_df['score'] = similarity_scores
    rec_tracks_df.sort_values(by=['score', 'popularity'], ascending=False, inplace=True)

    return rec_tracks_df



def make_track_URIs(track_ids):
    ### reformats track ids as track URIs
    ### need text spotify:track: in front of each ID to use in Spotify
    track_URIs = []
    for track_id in track_ids:
        uri = 'spotify:track:'+ track_id
        track_URIs.append(uri)
    return track_URIs

def create_playlist_file(track_ids, og_track_id, name):
    
    ### creates text file of Spotify URIs
    track_list = og_track_id.values.tolist() + track_ids.values.tolist()
    track_URIs = make_track_URIs(track_list)
    ### write URIs to text file
    playlist = open(fr'../playlists/playlist_{name}.txt','w')
    playlist.writelines('%s\n' % track for track in track_URIs) 
    playlist.close()
    pass

def display_playlist(playlist_tracks):
    ### displays playlist track name, artist, album
    tracks_dict = sp.tracks(playlist_tracks)['tracks']
    playlist_info = []
    for i in range(len(playlist_tracks)):
        track = [
            tracks_dict[i]['name'], 
            tracks_dict[i]['artists'][0]['name'],
            tracks_dict[i]['album']['name']
            ]
        playlist_info.append(track)
    
    playlist_df = pd.DataFrame(playlist_info, columns=['Title', 'Artist', 'Album'] )
    ### start index at 1
    playlist_df.index = np.arange(1,len(playlist_df)+1)
    return playlist_df


def top_recommended_tracks(results, num_tracks):
    
    top_half = results[results['score'] >= results['score'].median()].copy()

    top_half.sort_values(by='popularity', ascending=False, inplace=True)
    
    top_tracks = top_half[:num_tracks]
    
    return top_tracks

In [61]:
# results = track_reccommender(df, track, include_pop=False)
# results


track_name           object
artist               object
track_id             object
popularity          float64
danceability        float64
energy              float64
loudness            float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
dtype: object

In [94]:
scorer_choice = 'Cosine Similarity'
results = pop_track_reccommender2(df, track, scorer= scorer_choice)
results

NameError: name 'similarity_matrix' is not defined

In [91]:
top = top_recommended_tracks(results, 15)
top

,track_name,artist,popularity,track_id,genre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,score
0,Forever After All,Luke Combs,86.0,6IBcOGPsniK3Pso1wHIhew,country,0.487,0.650,-5.195,0.0253,0.191000,0.000000,0.0933,0.456,151.964,0.975005
11,"10,000 Hours (with Justin Bieber)",Dan + Shay,85.0,2wrJq5XKLnmhRXHIAf9xBa,country,0.654,0.630,-4.644,0.0259,0.153000,0.000000,0.1110,0.430,89.991,0.956894
66,Fortunate Son,Creedence Clearwater Revival,81.0,4BP3uh0hFLFRb5cjsgLqDh,country,0.640,0.663,-7.516,0.0374,0.201000,0.008060,0.1520,0.663,132.770,0.960292
2,One Of Them Girls,Lee Brice,80.0,14GwnOeC9qYEKEA6uOZepa,country,0.643,0.790,-6.079,0.0462,0.294000,0.000000,0.1070,0.800,95.987,0.957461
10,I Hope,Gabby Barrett,79.0,23T0OX7QOiIUFShSzbJ5Uo,country,0.466,0.536,-6.227,0.0429,0.095100,0.000000,0.1140,0.377,75.998,0.979566
116,More Than a Feeling,Boston,79.0,1QEEqeFIZktqIpPI4jSVSF,country,0.377,0.682,-8.039,0.0299,0.000894,0.002170,0.0504,0.288,108.736,0.985829
16,Beer Never Broke My Heart,Luke Combs,78.0,7aEtlGHoiPAfRB084NiDmx,country,0.538,0.863,-4.483,0.0481,0.013900,0.000000,0.0874,0.637,77.001,0.998932
180,Bad Moon Rising,Creedence Clearwater Revival,78.0,20OFwXhEXf12DzwXmaV7fj,country,0.508,0.774,-5.973,0.0321,0.058600,0.000005,0.0630,0.942,178.946,0.961241
18,When It Rains It Pours,Luke Combs,78.0,1mMLMZYXkMueg65jRRWG1l,country,0.551,0.801,-5.069,0.0303,0.013000,0.000006,0.2650,0.625,128.027,0.996769
22,Hurricane,Luke Combs,77.0,6xHI9KjUjYT0FPtGO8Mxa1,country,0.464,0.813,-6.185,0.0416,0.015300,0.000000,0.2540,0.515,75.977,0.999678


In [85]:
display_playlist(top['track_id'])

,Title,Artist,Album
1,Forever After All,Luke Combs,What You See Ain't Always What You Get (Deluxe...
2,"10,000 Hours (with Justin Bieber)",Dan + Shay,"10,000 Hours (with Justin Bieber)"
3,Fortunate Son,Creedence Clearwater Revival,Willy And The Poor Boys
4,One Of Them Girls,Lee Brice,One Of Them Girls
5,I Hope,Gabby Barrett,Goldmine
6,More Than a Feeling,Boston,Boston
7,Beer Never Broke My Heart,Luke Combs,What You See Is What You Get
8,Bad Moon Rising,Creedence Clearwater Revival,Green River
9,When It Rains It Pours,Luke Combs,This One's for You
10,Hurricane,Luke Combs,This One's for You


In [52]:
top_half = results[results['score'] >= results['score'].median()].copy()

top_half.sort_values(by='popularity', ascending=False, inplace=True)

In [53]:
top_half[:15]

,track_name,artist,popularity,track_id,genre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,score
0,Forever After All,Luke Combs,86.0,6IBcOGPsniK3Pso1wHIhew,country,0.487,0.650,-5.195,0.0253,0.191000,0.000000,0.0933,0.456,151.964,0.975005
11,"10,000 Hours (with Justin Bieber)",Dan + Shay,85.0,2wrJq5XKLnmhRXHIAf9xBa,country,0.654,0.630,-4.644,0.0259,0.153000,0.000000,0.1110,0.430,89.991,0.956894
66,Fortunate Son,Creedence Clearwater Revival,81.0,4BP3uh0hFLFRb5cjsgLqDh,country,0.640,0.663,-7.516,0.0374,0.201000,0.008060,0.1520,0.663,132.770,0.960292
2,One Of Them Girls,Lee Brice,80.0,14GwnOeC9qYEKEA6uOZepa,country,0.643,0.790,-6.079,0.0462,0.294000,0.000000,0.1070,0.800,95.987,0.957461
10,I Hope,Gabby Barrett,79.0,23T0OX7QOiIUFShSzbJ5Uo,country,0.466,0.536,-6.227,0.0429,0.095100,0.000000,0.1140,0.377,75.998,0.979566
116,More Than a Feeling,Boston,79.0,1QEEqeFIZktqIpPI4jSVSF,country,0.377,0.682,-8.039,0.0299,0.000894,0.002170,0.0504,0.288,108.736,0.985829
16,Beer Never Broke My Heart,Luke Combs,78.0,7aEtlGHoiPAfRB084NiDmx,country,0.538,0.863,-4.483,0.0481,0.013900,0.000000,0.0874,0.637,77.001,0.998932
180,Bad Moon Rising,Creedence Clearwater Revival,78.0,20OFwXhEXf12DzwXmaV7fj,country,0.508,0.774,-5.973,0.0321,0.058600,0.000005,0.0630,0.942,178.946,0.961241
18,When It Rains It Pours,Luke Combs,78.0,1mMLMZYXkMueg65jRRWG1l,country,0.551,0.801,-5.069,0.0303,0.013000,0.000006,0.2650,0.625,128.027,0.996769
22,Hurricane,Luke Combs,77.0,6xHI9KjUjYT0FPtGO8Mxa1,country,0.464,0.813,-6.185,0.0416,0.015300,0.000000,0.2540,0.515,75.977,0.999678


In [70]:
# results['track_id']
# top_half[:15]['track_id'].values

array(['6IBcOGPsniK3Pso1wHIhew', '2wrJq5XKLnmhRXHIAf9xBa',
       '4BP3uh0hFLFRb5cjsgLqDh', '14GwnOeC9qYEKEA6uOZepa',
       '23T0OX7QOiIUFShSzbJ5Uo', '1QEEqeFIZktqIpPI4jSVSF',
       '7aEtlGHoiPAfRB084NiDmx', '20OFwXhEXf12DzwXmaV7fj',
       '1mMLMZYXkMueg65jRRWG1l', '6xHI9KjUjYT0FPtGO8Mxa1',
       '6vC90OOjZR165Hw8CpsqEm', '698eQRku24PIYPQPHItKlA',
       '1D7cfiC5mxqHfTCcOiRBej', '0nYvjcSlCgjcwogQAwIwNp',
       '70YvYr2hGlS01bKRIho1HM'], dtype=object)

In [76]:
# track_ids = track['track_id'].values.tolist() + top_half[:15]['track_id'].values.tolist()
# track_ids
create_playlist_file(track_ids=top_half[:15]['track_id'], og_track_id=track['track_id'], name='boston-country')

In [57]:
display_playlist(top_half[:15]['track_id'])

,Title,Artist,Album
1,Forever After All,Luke Combs,What You See Ain't Always What You Get (Deluxe...
2,"10,000 Hours (with Justin Bieber)",Dan + Shay,"10,000 Hours (with Justin Bieber)"
3,Fortunate Son,Creedence Clearwater Revival,Willy And The Poor Boys
4,One Of Them Girls,Lee Brice,One Of Them Girls
5,I Hope,Gabby Barrett,Goldmine
6,More Than a Feeling,Boston,Boston
7,Beer Never Broke My Heart,Luke Combs,What You See Is What You Get
8,Bad Moon Rising,Creedence Clearwater Revival,Green River
9,When It Rains It Pours,Luke Combs,This One's for You
10,Hurricane,Luke Combs,This One's for You


In [47]:
results['score'].describe()

count    2001.000000
mean        0.907891
std         0.118206
min         0.352937
25%         0.885545
50%         0.955774
75%         0.983660
max         1.000000
Name: score, dtype: float64

In [51]:
cols = ['artist_name','score', 'popularity', 'danceability',
       'energy', 'valence', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'loudness','tempo',
       'track_id']
results = results[cols]
results

,artist_name,score,popularity,danceability,energy,valence,speechiness,acousticness,instrumentalness,liveness,loudness,tempo,track_id
9295,Cardi B,1.000000,0.97,0.935,0.454,0.357,0.375,0.019400,0.000000,0.0824,-7.509,133.073,4Oun2ylbjFKMPTiaSbbCih
48,21 Savage,0.997328,0.84,0.884,0.346,0.376,0.351,0.015100,0.000007,0.0871,-8.228,75.016,2fQrGHiQOvpL9UgPvtYy6G
155,Future,0.996250,0.82,0.833,0.434,0.281,0.431,0.010200,0.021900,0.1650,-8.795,150.062,0VgkVdmE4gld66l8iyGjgx
1389,XXXTENTACION,0.995345,0.67,0.606,0.349,0.239,0.198,0.076200,0.071100,0.1000,-10.032,180.055,5b6zjsBrZCqe6RFycLz2tc
194,Drake,0.994348,0.75,0.766,0.442,0.390,0.356,0.000107,0.000061,0.1110,-8.558,201.800,4Kz4RdRCceaA9VgTqBhBfa
243,21 Savage,0.993051,0.77,0.885,0.520,0.306,0.359,0.000244,0.000001,0.4540,-8.353,84.022,2wOXxtHZgRkkrkEbKLzzqs
448,Future,0.992899,0.72,0.730,0.465,0.367,0.278,0.011700,0.000000,0.2800,-6.290,148.028,4VMiNOpnjRwSCwmHaUfOMM
166,ScHoolboy Q,0.992471,0.77,0.768,0.471,0.405,0.259,0.020100,0.000000,0.2680,-8.406,131.023,4LmAnpjlhWTahvRkYR8xJa
1553,Fat Nick,0.992404,0.68,0.723,0.459,0.313,0.239,0.000960,0.000000,0.1100,-10.259,139.972,3Bo3lNVQTtZYMCoItX8dKW
1310,Future,0.992364,0.71,0.727,0.431,0.223,0.395,0.011100,0.016600,0.5060,-12.007,150.005,6DB2KOEwHnjkgEnBt5SdeJ


In [52]:
make_track_URIs(results['track_id'])

['spotify:track:4Oun2ylbjFKMPTiaSbbCih',
 'spotify:track:2fQrGHiQOvpL9UgPvtYy6G',
 'spotify:track:0VgkVdmE4gld66l8iyGjgx',
 'spotify:track:5b6zjsBrZCqe6RFycLz2tc',
 'spotify:track:4Kz4RdRCceaA9VgTqBhBfa',
 'spotify:track:2wOXxtHZgRkkrkEbKLzzqs',
 'spotify:track:4VMiNOpnjRwSCwmHaUfOMM',
 'spotify:track:4LmAnpjlhWTahvRkYR8xJa',
 'spotify:track:3Bo3lNVQTtZYMCoItX8dKW',
 'spotify:track:6DB2KOEwHnjkgEnBt5SdeJ',
 'spotify:track:2Grb4G6t9VIqo6moKUloom',
 'spotify:track:2b4SSorCTQ2VzmllaeWuuT',
 'spotify:track:3mvYQKm8h6M5K5h0nVPY9S',
 'spotify:track:2bjwRfXMk4uRgOD9IBYl9h',
 'spotify:track:4JsKaTag0OHXibnqYcDfIA']

In [28]:
create_playlist_file(results['track_id'], 'WAP-country')

In [27]:
# display_playlist(make_track_URIs(results['track_id']))

,Title,Artist,Album
1,WAP (feat. Megan Thee Stallion),Cardi B,WAP (feat. Megan Thee Stallion)
2,One Of Them Girls,Lee Brice,One Of Them Girls
3,More Than My Hometown,Morgan Wallen,More Than My Hometown
4,Beer Never Broke My Heart,Luke Combs,What You See Is What You Get
5,When It Rains It Pours,Luke Combs,This One's for You
6,Whiskey Glasses,Morgan Wallen,If I Know Me
7,Love You Like I Used To,Russell Dickerson,Love You Like I Used To
8,One Man Band,Old Dominion,Old Dominion
9,Chasin' You,Morgan Wallen,If I Know Me
10,Pretty Heart,Parker McCollum,Pretty Heart


## Try it without Popularity <a class="anchor" id="topic-2"></a>
<hr/>

In [12]:
# results_nopop = track_reccommender(df, track, include_pop=False)

In [13]:
# results_nopop

,artist_name,track_id,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
3113,The Sugarhill Gang,0FWhGmPVxLI6jOVF0wjALa,0.56,0.465000,0.900,0.845,0.000000,0.0724,-7.306,0.257,111.187,0.914
3191,Ice Cube,4r13PlX7hKjNxdyq5ukR5A,0.63,0.000917,0.946,0.857,0.000000,0.0525,-4.498,0.159,99.880,0.882
3627,$uicideBoy$,6OB7Xp69uSWA4vDwcKgEDA,0.60,0.119000,0.905,0.817,0.000000,0.1500,-4.260,0.397,140.083,0.800
4543,Noname,1XiKRU9Dg7QPlGRSWbfXgo,0.57,0.091000,0.891,0.827,0.000002,0.2210,-6.139,0.351,100.987,0.861
4749,G-Eazy,6uz0z5oMeC8QWfTHeR5qlF,0.56,0.241000,0.838,0.781,0.000000,0.1050,-8.249,0.450,135.941,0.913
4854,Lil Pump,6ox6GMJt3WP2QghksDa6UO,0.60,0.026300,0.917,0.760,0.000000,0.0976,-3.290,0.380,136.014,0.813
4922,Key Glock,3RHOFe6m1g0nCZ7CClVKn0,0.54,0.001340,0.935,0.772,0.000028,0.1100,-8.773,0.354,139.999,0.875
5873,Young Dolph,0xSLJxGbnhA6FHtar3CEmc,0.51,0.106000,0.866,0.947,0.000000,0.6310,-5.365,0.272,132.013,0.813
6969,2 LIVE CREW,0ioFIQXbTUaliQpsJyX4zN,0.49,0.001070,0.895,0.900,0.000661,0.2390,-8.905,0.195,131.671,0.896
7329,EPMD,0f1yzIIGD8k32DNVeEn6jb,0.50,0.089000,0.882,0.814,0.000000,0.1600,-5.214,0.424,99.689,0.854


In [14]:
# display_playlist(make_track_URIs(results_nopop['track_id']))

,Title,Artist,Album
1,Rapper's Delight,The Sugarhill Gang,Rhino Hi-Five: The Sugarhill Gang
2,You Can Do It - Feat. Mack 10 And Ms Toi,Ice Cube,War & Peace Vol. 2 (The Peace Disc)
3,HUNG UP ON THE COME UP,$uicideBoy$,HUNG UP ON THE COME UP
4,Blaxploitation,Noname,Room 25
5,Loaded (Bonus Track) [feat. DJ Carnage],G-Eazy,Must Be Nice
6,Smoke My Dope (feat. Smokepurpp),Lil Pump,Lil Pump
7,Dope,Key Glock,Glockoma
8,What's the Deal,Young Dolph,Thinking Out Loud
9,Hoochie Mama,2 LIVE CREW,Goes To the Movies: Decade of Hits
10,You Gots To Chill,EPMD,Strictly Business


In [39]:

Y = np.array([
    [1,2,3],
    [6,4,5],
    [9,7,8]
]
)

s = cosine_similarity(Y, Y)
s

array([[1.        , 0.88326011, 0.90184723],
       [0.88326011, 1.        , 0.99819089],
       [0.90184723, 0.99819089, 1.        ]])

In [38]:
s[1]

array([0.88326011, 1.        , 0.99819089])

In [77]:
playlist_file = open('../playlists/playlist_boston-country.txt', 'r+')

text = playlist_file.read().splitlines()

In [78]:
display_playlist(text)

,Title,Artist,Album
1,Boston,Dalton & the Sheriffs,Luckier by Half
2,Forever After All,Luke Combs,What You See Ain't Always What You Get (Deluxe...
3,"10,000 Hours (with Justin Bieber)",Dan + Shay,"10,000 Hours (with Justin Bieber)"
4,Fortunate Son,Creedence Clearwater Revival,Willy And The Poor Boys
5,One Of Them Girls,Lee Brice,One Of Them Girls
6,I Hope,Gabby Barrett,Goldmine
7,More Than a Feeling,Boston,Boston
8,Beer Never Broke My Heart,Luke Combs,What You See Is What You Get
9,Bad Moon Rising,Creedence Clearwater Revival,Green River
10,When It Rains It Pours,Luke Combs,This One's for You


In [90]:
playlist_file = open('../playlists/playlist_boston-country.txt', 'r+')

text = playlist_file.read().splitlines()
text

['spotify:track:4HJ7mSMtHAdU55lLjGE4zW',
 'spotify:track:6IBcOGPsniK3Pso1wHIhew',
 'spotify:track:2wrJq5XKLnmhRXHIAf9xBa',
 'spotify:track:4BP3uh0hFLFRb5cjsgLqDh',
 'spotify:track:14GwnOeC9qYEKEA6uOZepa',
 'spotify:track:23T0OX7QOiIUFShSzbJ5Uo',
 'spotify:track:1QEEqeFIZktqIpPI4jSVSF',
 'spotify:track:7aEtlGHoiPAfRB084NiDmx',
 'spotify:track:20OFwXhEXf12DzwXmaV7fj',
 'spotify:track:1mMLMZYXkMueg65jRRWG1l',
 'spotify:track:6xHI9KjUjYT0FPtGO8Mxa1',
 'spotify:track:6vC90OOjZR165Hw8CpsqEm',
 'spotify:track:698eQRku24PIYPQPHItKlA',
 'spotify:track:1D7cfiC5mxqHfTCcOiRBej',
 'spotify:track:0nYvjcSlCgjcwogQAwIwNp',
 'spotify:track:70YvYr2hGlS01bKRIho1HM']